In [26]:
!pip install langchain
!pip install langchain_chroma
!pip install langchain_community
!pip install langchain_core
!pip install langchain_openai
!pip install langchain_text_splitters
!pip install langgraph
!pip install amazon-textract-textractor
import bs4
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain.tools.retriever import create_retriever_tool
from langchain_chroma import Chroma
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.prebuilt import create_react_agent
%pip install --upgrade --quiet  boto3 langchain-openai tiktoken python-dotenv
%pip install --upgrade --quiet  "amazon-textract-caller>=0.2.0"


In [27]:
from langchain_community.document_loaders import AmazonTextractPDFLoader

In [30]:
pip install pypdf

In [31]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("/content/Do Epic Shit (Warikoo, Ankur) (Z-Library).pdf")

In [33]:
pages = loader.load()
pages

[Document(metadata={'source': '/content/Do Epic Shit (Warikoo, Ankur) (Z-Library).pdf', 'page': 0}, page_content=''),
 Document(metadata={'source': '/content/Do Epic Shit (Warikoo, Ankur) (Z-Library).pdf', 'page': 1}, page_content='ABOUT THE AUTHOR\nAnkur W arikoo  is an entrepreneur and content creator whose deep, witty and brutally honest\nthoughts on success and failure, money and investin g, self-awareness and personal\nrelationships have made him one of India’s top pers onal brands.\nIn his first book, Ankur puts together the key idea s that have fuelled his journey – one that\nbegan with him wanting to be a space engineer and e nded with him creating content that has\nbeen seen and read by millions.\nHis thoughts range from the importance of creating habits for long-term success to the\nfoundations of money management, from embracing and  accepting failure to the real truth\nabout learning empathy.\nThis is a book to be read, and reread, a book whose  lines you will underline and

In [35]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


In [36]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
if not os.environ.get("LANGCHAIN_API_KEY"):
    os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

··········


In [37]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo")

In [39]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(pages)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()

In [40]:
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [41]:
from langchain.tools.retriever import create_retriever_tool

tool = create_retriever_tool(
    retriever,
    "book_retriever",
    "Searches and returns excerpts from the Do Epic Shit book.",
)
tools = [tool]

In [44]:
tool.invoke("Before you assume, try this crazy thing")

'People aren’t good to you? Maybe they are trying to be good to\nthemselves as well. Why not ask?\nWant to talk to your parents? Maybe they want to, as well . Why not\nask?\nAsking is the best thing to do, before making assumptions.\nIt either validates your assumption or you get to underst and people\nbetter. Either way, you get to be a better communicato r.\nWin-win :)\nMost important skills today that are hardly taught:\nHumour\n\nBreaking your limits without even expecting to is the po werful way to\nsee how much more you are capable of.\nOthers’ success will generate massive\nself-doubt every morning when you\nget up.\nGet up anyway.\n. . .\nA year from now, you will wish you had started toda y.\nStart today.\nTime goes away and leaves us with only one of these two t hings:\nregret or results.\n\nNeither of this will happen if you stay where you are.\nMovement brings momentum.\nStagnation amplifies sadness.\nThe best mental model for taking tough decisions in  life?\n1. Ask yourse

In [45]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(llm, tools)

In [47]:
from langchain_core.messages import AIMessage, HumanMessage
query = "What is this book is about?"

for s in agent_executor.stream(
    {"messages": [HumanMessage(content=query)]},
):
    print(s)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_XZsmDDCXoIgJbGy2mg7PygYr', 'function': {'arguments': '{"query":"overview"}', 'name': 'book_retriever'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 69, 'total_tokens': 85}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-0189002d-84ce-4b1b-952f-416cccd1560b-0', tool_calls=[{'name': 'book_retriever', 'args': {'query': 'overview'}, 'id': 'call_XZsmDDCXoIgJbGy2mg7PygYr'}], usage_metadata={'input_tokens': 69, 'output_tokens': 16, 'total_tokens': 85})]}}
----
{'tools': {'messages': [ToolMessage(content='Extra\n\nTINTRODUCTION\nhis book may very well turn out to be the most useless book\nyou will ever buy. Because nothing in this book is somethi ng\nyou don’t know of.\nThis book is not going to be a revelation. It is meant to be a\nreminder. A reminder of how life happens 

In [48]:
from langgraph.checkpoint.sqlite import SqliteSaver

memory = SqliteSaver.from_conn_string(":memory:")

agent_executor = create_react_agent(llm, tools, checkpointer=memory)

In [49]:
config = {"configurable": {"thread_id": "abc123"}}

for s in agent_executor.stream(
    {"messages": [HumanMessage(content="Hi! I'm mohit")]}, config=config
):
    print(s)
    print("----")

{'agent': {'messages': [AIMessage(content='Hello Mohit! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 69, 'total_tokens': 81}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-4e62285b-65c5-493f-a20d-430aeb91ae1e-0', usage_metadata={'input_tokens': 69, 'output_tokens': 12, 'total_tokens': 81})]}}
----


In [62]:
query = "What according to the book are common ways of doing it? redo the search"

for s in agent_executor.stream(
    {"messages": [HumanMessage(content=query)]}, config=config
):
    print(s)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_PXutKtR7eIFDuOv04jnpML8v', 'function': {'arguments': '{"query":"common ways of doing it"}', 'name': 'book_retriever'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 2624, 'total_tokens': 2644}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-39cc6b18-7dc4-42bd-b329-86e1819cd6a3-0', tool_calls=[{'name': 'book_retriever', 'args': {'query': 'common ways of doing it'}, 'id': 'call_PXutKtR7eIFDuOv04jnpML8v'}], usage_metadata={'input_tokens': 2624, 'output_tokens': 20, 'total_tokens': 2644})]}}
----
{'tools': {'messages': [ToolMessage(content='P ART 2  \nHABITS\n\nThe easiest way to learn from mistakes is to read b ooks.\nThe next option is to commit them yourself.\nBooks are a great way to learn from mistakes someone else h as\nmade over their lifetime!\nWhy would you want 